In [5]:
import pymc3 as pm
import arviz as az
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import sys
from time import time
np.random.seed(42) 
#You will get a warning for the mkl package if you are running on M1-based machines

In [6]:
path = os.getcwd()
files = os.listdir(path)
df = {}
xlsx = pd.ExcelFile('ExpData.xlsx')
df = pd.read_excel(xlsx, sheet_name=None)
data = [[k,v] for k,v in df.items()] #k is the sheet name, v is the pandas df

i = 7
X, y = data[i][1]['ShearRate'], data[i][1]['ShearStress']
y = np.log10(y)

kk = float((np.log10(np.max(10**y))+np.log10(np.min(10**y)))/2)
# kk = 0.
y = y - kk
X, y = np.array(X), np.array(y)

In [7]:
def TCC(sigma_y, g_ct, k, g_c, X):
    Sigma = np.log10(sigma_y*(1+(X/g_ct)**0.5) + X*k*(1+(X/g_c)**2)**-0.5)
    return Sigma


# Intial Conditions and Parameters
sigma_y = 1.
g_ct = 1.
k = 1.
g_c = 1.

In [8]:
t0 = time()
with pm.Model() as model_g:
    a = pm.Uniform('a', 0., 20.)
    b = pm.Uniform('b', 0., 20.)
    c = pm.Uniform('c', -10., 10.)
    d = pm.Uniform('d', 0., 20.)
    ϵ = pm.HalfNormal('ϵ', sigma=5)    # prior for our estimated standard deviation of the error

    T_pred = pm.Normal('T_pred', mu=TCC(a, b, c, d, X), sd=ϵ, observed=y)  # bringing it all together


    trace_g = pm.sample(2000, tune=1000,cores=2)  # Explore and Sample the Parameter Space!

runtime = np.round(time()-t0,2)
print('\nBIC computation time: {} seconds'.format(runtime))

/Users/miladsa/tf/envs/pymc-dev-py39/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [ϵ, d, c, b, a]
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


/Users/miladsa/tf/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/miladsa/tf/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 16 seconds.
There were 28 divergences after tuning. Increase `target_accept` or reparameterize.
There were 172 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.



BIC computation time: 17.95 seconds


In [9]:
az.summary(trace_g)

Got error No model on context stack. trying to find log_likelihood in translation.
/Users/miladsa/tf/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,0.002,0.001,0.000,0.003,0.000,0.000,333.0,202.0,1.01
b,0.000,0.000,0.000,0.000,0.000,0.000,340.0,202.0,1.01
c,1.323,0.020,1.288,1.361,0.001,0.001,455.0,1447.0,1.01
d,16.683,0.409,15.931,17.483,0.015,0.011,728.0,963.0,1.01
ϵ,0.017,0.002,0.013,0.020,0.000,0.000,159.0,80.0,1.01
